# Demo of overlay of comparison stars

## Go to "Cell → Run all" then scroll to the bottom of the page

### Be patient while the image is downloaded

In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
from astrowidgets import ImageWidget, __version__
from stellarphot.visualization.comparison_functions import *
import ipywidgets as ipw
import numpy as np
from astropy.coordinates import SkyCoord
import functools
from astropy.table import Table

/Users/mattcraig/miniconda3/envs/stellarphot/lib/python3.8/site-packages/ginga/cmap.py:13317: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  for name in _cm.cmap_d:


In [4]:
print(__version__)

0.1.1


In [5]:
directory_with_images = '.'
#This could be any image from roughly the middle of the sequence.
sample_image_for_finding_stars = 'TIC_470127886.01-S001-R001-C001-rp.fit'
object_of_interest = 'TYC 4612-401-1'

In [6]:
df, RD, ra, dec, RD['coords'] = read_file('gaia_stars_1596472748.radec')

In [7]:
ccd, ccdx, ccdy, vsx, ra, dec, vsx['coords'], vsx_names = set_up(object_of_interest, directory_with_images, sample_image_for_finding_stars)

['_q', 'OID', 'n_OID', 'Name', 'V', 'Type', 'l_max', 'max', 'u_max', 'n_max', 'f_min', 'l_min', 'min', 'u_min', 'n_min', 'l_Period', 'Period', 'u_Period', 'RAJ2000', 'DEJ2000', 'coords']


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [8]:
apass, v_angle, RD_angle = match(ccd, RD, vsx)

In [9]:
Cmag = 11.3

In [10]:
apass_good_coord, good_stars = mag_scale(Cmag, apass, v_angle, RD_angle, 
                                         brighter_dmag=3, 
                                         dimmer_dmag=5)

In [11]:
ent = in_field(apass_good_coord, ccd, ccdx, ccdy, apass, good_stars)

In [12]:
iw = ImageWidget()
out = ipw.Output()
bind_map = iw._viewer.get_bindmap()
gvc = iw._viewer.get_canvas()
bind_map.map_event(None, ('shift',), 'ms_left', 'cursor')
gvc.add_callback('cursor-down', wrap(iw, out))

In [13]:
header = ipw.HTML(value="""
<h2>Scroll to pan, use +/- keys to zoom</h2>
<h3>Left click to exclude star as target or comp. Click again to include.</h3>
""")

legend = ipw.HTML(value="""
<h3>Green circles -- Gaia stars within 2.5 arcmin of target</h3>
<h3>Blue circles -- APASS stars within 1 mag of target</h3>
<h3>Red circles -- VSX variables</h3>
<h3>Red × -- Exclude as target or comp</h3>
""")

In [14]:
# make_markers needs a local file for now...
ccd.write('image_file.fits', overwrite=True)
make_markers(iw, directory_with_images, 'image_file.fits', RD, vsx, ent, object_of_interest)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [15]:
RD

RA,Dec,a,b,Mag,coords
,,,,,"deg,deg"
str12,str14,int64,int64,float64,object
22:04:28.286,81:33:57.442,0,1,10.707,"331.1178583333333,81.5659561111111"
22:04:36.467,81:33:16.436,0,0,15.867,"331.1519458333333,81.55456555555556"
22:04:39.668,81:34:56.136,0,0,13.502,"331.16528333333326,81.58225999999999"
22:04:22.390,81:32:49.766,0,0,14.009,"331.09329166666663,81.54715722222222"
22:04:54.968,81:33:15.172,0,0,15.85,"331.2290333333333,81.55421444444444"
22:03:51.438,81:33:30.800,0,0,14.909,"330.964325,81.55855555555556"
22:03:34.951,81:34:24.798,0,0,14.457,"330.89562916666665,81.573555"
22:03:26.726,81:33:22.258,0,0,15.341,"330.86135833333327,81.55618277777778"


In [16]:
box = ipw.VBox()
inner_box = ipw.HBox()
inner_box.children = [iw, legend]
box.children = [header, inner_box]
box

In [26]:
vsx_names

PMAK V70
MASTER OT J220220.89+814532.5
WISE J221032.6+815314


In [ ]:
all_table = iw.get_markers(marker_name = 'all')
elims = np.array([name.startswith('elim') for name in all_table['marker name']])
elim_table = all_table[elims]
comp_table = all_table[~elims]

In [ ]:
aperture_table = Table.read('comp_stars_with_good_id.fits')
aperture_table

In [ ]:
target = RD[0]
iw.marker = {'color': 'yellow', 'radius': 268, 'type': 'circle'}
iw.add_markers(target, skycoord_colname='coords',use_skycoord=True, marker_name='target')
iw.stop_marking

In [ ]:
comp_table[13]

In [ ]:
index, d2d, d3d = elim_table['coord'].match_to_catalog_sky(comp_table['coord'])
comp_table.remove_rows(index)

In [ ]:
target_coord = RD['coords'][0]

# Calculate how far each is from target
comp_table['separation'] = target_coord.separation(comp_table['coord'])

# Change that comparison that is actually a variable to a variable 
comp_table['marker name'][13] = 'VSX'

# Add dummy column for sorting in the order we want
comp_table['sort'] = np.zeros(len(comp_table))

# Set sort order
apass = comp_table['marker name'] == 'APASS comparison'
vsx = comp_table['marker name'] == 'VSX'
tess = comp_table['marker name'] == 'TESS Targets'

comp_table['sort'][apass] = 2
comp_table['sort'][vsx] = 1
comp_table['sort'][tess] = 0

# Sort the table
comp_table.sort(['sort', 'separation'])

# Assign the IDs
comp_table['new_id'] = range(1, len(comp_table) + 1)

# Delete the columns we don't need
#del comp_table['separation'], comp_table['sort']

In [ ]:
new_vsx = comp_table[10]
iw.marker = {'color': 'blue', 'radius': 20, 'type': 'circle'}
iw.add_markers(Table(data=[[new_vsx['x']], [new_vsx['y']]], names=['x', 'y']))
iw.stop_marking

In [ ]:
comp_table

In [ ]:
for star in comp_table:
    id = star['new_id']
    if star['marker name'] == 'TESS Targets': 
        iw._marker = functools.partial(iw.dc.Text, text=f'T{id}', fontsize=20, fontscale=False, color='green')
        iw.add_markers(Table(data=[[star['x']+20], [star['y']-20]], names=['x', 'y']))

    elif star['marker name'] == 'APASS comparison':
        iw._marker = functools.partial(iw.dc.Text, text=f'C{id}', fontsize=20, fontscale=False, color='red')
        iw.add_markers(Table(data=[[star['x']+20], [star['y']-20]], names=['x', 'y']))

    else:
        iw._marker = functools.partial(iw.dc.Text, text=f'V{id}', fontsize=20, fontscale=False, color='blue')
        iw.add_markers(Table(data=[[star['x']+20], [star['y']-20]], names=['x', 'y']))

iw.stop_marking()

In [ ]:
comp_table.write('new_comp_locations.fits')

In [ ]:
iw.stop_marking()

In [ ]:
iw.save('TIC470127886-01_20200627_Paul-P-Feder-0.4m_rp_field-zoom.png')